# **Introduction**

The goal of this model is to be able to generate a synopsis for a SpongeBob episode based on a given title. I have collected data from the [SpongeBob wiki](https://spongebob.fandom.com/wiki/Encyclopedia_SpongeBobia) and paired episode titles with their given sumamries. Hopefully, this data can help fine-tune a model to be able to output SpongeBob episodes similar in style to how the sumarries are written on the SpongeBob wiki.

# **Model Creation**

### Preparations

Before I can fine-tune the model, I have to first install the needed packages and ensure the data is ready for GPT3 to use. I will be using an OpenAI tool that makes suggestions on how to optomize my data for model use and will be implementing the suggestions in order to hopefully optomize the model's performance.

Additionally, I needed to set the OpenAI API key. The code of me doing this has been excluded, but if you wish to run this code yourself, make sure you set the OpenAI API key environment variable to your own API key.

In [3]:
!pip install --upgrade openai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 42 kB 750 kB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 163 kB 9.0 MB/s 
  Created wheel for openai: filename=openai-0.20.0-py3-none-any.whl size=54118 sha256=9548c62ed9824690df4fd3f8bfe56b8eaf4e8f5fef65c0c394ead8245ca162c3
  Stored in directory: /root/.cache/pip/wheels/71/8d/9b/e28529ec53123e0279208f99148d4661232120d78cb866839b
Successfully built openai


In [4]:
!openai tools fine_tunes.prepare_data -f spongebob-data.jsonl

Logging requires wandb to be installed. Run `pip install wandb`.
Analyzing...

- Your file contains 509 prompt-completion pairs
- There are 2 examples that are very long. These are rows: [191, 334]
For conditional generation, and for classification the examples shouldn't be longer than 2048 tokens.
- Your data does not contain a common separator at the end of your prompts. Having a separator string appended to the end of the prompt makes it clearer to the fine-tuned model where the completion should begin. See https://beta.openai.com/docs/guides/fine-tuning/preparing-your-dataset for more detail and examples. If you intend to do open-ended generation, then you should leave the prompts empty
- Your data does not contain a common ending at the end of your completions. Having a common ending string appended to the end of the completion makes it clearer to the fine-tuned model where the completion should end. See https://beta.openai.com/docs/guides/fine-tuning/preparing-your-dataset for mo

### Fine-Tuning

Next, I will be creating a fine-tuned model based off the aforementioned data, then I will run a test input in order to analyze whether the model works as intended.

In [18]:
!openai api fine_tunes.create -t "file-1rdrvuvGQMc5RsgaarrA19PA" -m "curie"

Logging requires wandb to be installed. Run `pip install wandb`.
Created fine-tune: ft-I32aL3nFEdZFzST4ZVniXmSm
Streaming events until fine-tuning is complete...

(Ctrl-C will interrupt the stream, but not cancel the fine-tune)
[2022-07-18 01:11:50] Created fine-tune: ft-I32aL3nFEdZFzST4ZVniXmSm
[2022-07-18 01:11:54] Fine-tune costs $4.19
[2022-07-18 01:11:55] Fine-tune enqueued. Queue number: 0
[2022-07-18 01:11:57] Fine-tune started
[2022-07-18 01:15:12] Completed epoch 1/4
[2022-07-18 01:17:32] Completed epoch 2/4
[2022-07-18 01:19:52] Completed epoch 3/4
[2022-07-18 01:22:12] Completed epoch 4/4
[2022-07-18 01:22:33] Uploaded model: curie:ft-personal-2022-07-18-01-22-33
[2022-07-18 01:22:35] Uploaded result file: file-fMW4AVlcvSBvhxoK7ClwOmow
[2022-07-18 01:22:35] Fine-tune succeeded

Job complete! Status: succeeded 🎉
Try out your fine-tuned model:

openai api completions.create -m curie:ft-personal-2022-07-18-01-22-33 -p <YOUR_PROMPT>


In [45]:
import openai

openai.Completion.create(
    model="curie:ft-personal-2022-07-18-01-22-33",
    prompt="Plankton Steals the Formula",
    max_tokens=1500,
    temperature=0.9,
    frequency_penalty=0.75,
    n=1
  )

<OpenAIObject text_completion id=cmpl-5VBnLmEeKQbAkE5ii5tlrDmHp6Nya at 0x7fd529ff6bf0> JSON: {
  "choices": [
    {
      "finish_reason": "length",
      "index": 0,
      "logprobs": null,
      "text": " -> Despite being on the run from the police and SpongeBob, the Chum Bucket thief Plankton manages to steal both the formula and the S.S. Krabby Patty secret formula, but his attempt fails when he trips over a piece of equipment. He ends up falling off of a bridge and lands back in his own kitchen, with Patrick. Plankton then realizes that this is his daughter Karen's (actual) birthday and gets an idea, leading to him leaving in his submarine with Karen as hostage along with both formulas.\nLater at the Krusty Krab, Mr. Krabs is happy he got back the old formula when Plankton returns to give him SpongeBob while holding Karen along with the two formulas until they escape via freeze ray attack thanks to sending Plankton flying into a refrigerator twice before pushing it away using a fo

As it is very much evident, this is not a very coherent episode at all. While the output starts off readable (although lacking a lot of consistency), the output slowly devolves into more confusing nonsense. Part of this has to do with the limited ability of GPT3 to create large outputs for text completion. While I was optomistic that fine-tuning on large episode summaries could potnetially help to overcome this, it is evident that this is definetly not the case.

Therefore, my next plan is to make the max token size a much smaller number in order to make the output more coherent. However, to complete the episode, I will continually pass the output as input to the Davinci model, where it will progressivly build more and more of the episode, until it either reaches its token limit or runs out of things to add. Essentially, this is like using the fine-tuned model to get ball rolling on what to create the SpongeBob episode around, then using pre-existing models to finish up the job.

In [46]:
# Creates a SpongeBob episode based off of the given title
def gen_ep(title):
  # Gets an initial output from the fine-tuned model
  ft_model_res = gen_new_output(title, "curie:ft-personal-2022-07-18-01-22-33", 250)
  res_text = ft_model_res["choices"][0]["text"][4:]

  # Initializes the necessary variables for the next step
  res_tokens = ft_model_res["usage"]["total_tokens"]
  prev_tokens = 0
  res_output = f"\"{title}\"\n{res_text}"

  # Continually makes calls to the Davinci model to add more to the episode until
  # either the max length is reached or the model ends the episode and cannot add more
  while res_tokens < 1750 and res_tokens - prev_tokens > 5:
    # Makes a new call to the model
    res = gen_new_output(res_output, "text-davinci-001", min(2000 - res_tokens, 350))

    # Updates the ongoing text and token variables
    res_output += res["choices"][0]["text"]
    prev_tokens = res_tokens
    res_tokens = res["usage"]["total_tokens"]

  # Returns the text of the episode
  return res_output


# When given an input, model name, and max tokens, returns the output from that model
def gen_new_output(input, model, tokens):
  return openai.Completion.create(
    model=model,
    prompt=input,
    max_tokens=tokens,
    temperature=0.9,
    frequency_penalty=0.75,
    n=1
  )

# Examples

In [48]:
# Example with character name in the title
print(gen_ep("Krabs Goes Broke"))
print("===============================")
# Example with common item on the show in the title
print(gen_ep("The Chum Monster"))
print("===============================")
# Example with nothing related to the show in the title
print(gen_ep("The New Zoo"))

"Krabs Goes Broke"
In the beginning of the episode, SpongeBob arrives at Mr. Krabs' Velma Cammobile to start his day at work. While helping Mr. Krabs open his safe with a crowbar, the crab tells SpongeBob that he needs to make sure no one borrows money from him during the next 24 hours. However, due to having spent most of his own funds on various things (including a kickback from Squidward's book rental), he can barely afford any food for himself for lunch.
It then shows Patrick buying all kinds of things at his convenience store without paying for them, such as 32 bags of sugar, a never-ending supply of coffee creamer and a bag of seaweed snacks without buying any actual food items. When Squidward objects and suggests that Patrick buys some hot dogs instead, the latter states he only has fifty cents and reveals that SpongeBob knows how to get that amount in less than twenty seconds in exchange for not telling anyone about this "secret."
Squidward suggests they go into business with e

# Conclusion

Ultimately, it's obvious that the second strategy of using smaller token sizes on each model call worked signficantly better than my original plan of generating an entire episode synopsis just from one use of the model. However, this is not to say that the results are flawless. The above examples show that, while the episode summaries generally read coherently, there are still many major issues in the plots that are developed that can leave you scratching your head on what is going on, as well as there being many awkwardly structured sentences that can be misunderstood at first. However, amidst all these issues, the generated episodes did show a good amount of creativity and interesting story telling at some parts.

What is definetely noticeable from the above examples is that the specificty of the title helps a lot in keeping it alligned with the topic. The first example of "Krabs Goes Broke" generated an episode that very much focuses on this idea, whereas the last example of "The New Zoo" has practically very little with a zoo. Perhaps re-tuning the model to take a brief sentence summary of an episode rather than the title could lead to much better results.

Overall, the idea of using a model to generate a cartoon episode could have implications in other uses of AI. For example, rather than generating SpongeBob episodes, one could perhaps fine-tune a model to give suggestions on what improvements a company should make to its product or what new and interesting coding projects one could pursue. While the output of my model wasn't perfect, it illustrates GTP3's ability to be creative and think of new ideas.